# Upper Grindelwald case study

### Imports

In [58]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
%matplotlib inline
fig_size = 12, 6
plt.rcParams['figure.figsize'] = (fig_size)  # Default plot size
import scipy.stats as stats
import numpy as np
import pandas as pd
import netCDF4
import salem
import copy

In [59]:
import oggm
from oggm import cfg
from oggm import workflow
from oggm import utils
from oggm import graphics
from oggm.core.models import massbalance, flowline
from oggm.core.preprocessing.inversion import invert_parabolic_bed

In [60]:
cfg.initialize()  # read the default parameter file for OGGM
lengths_data = pd.read_csv(
    utils.get_demo_file('grindelwald_lengths.csv'), index_col=0)

### Init glacier directory

In [61]:
bdir = '/Users/oberrauch/bac/raw_data/grindelwald/'
gdir = utils.GlacierDirectory('RGI50-11.01270', base_dir=bdir)

In [62]:
flowline.init_present_time_glacier(gdir)

### Mass-balance data

In [63]:
# We can build a mass-balance model based on
# these values and climate from HISTALP
mb_mod = massbalance.HistalpMassBalanceModel(gdir)

### Glen's flow law parameters

In [64]:
glen_a = cfg.A

## Try different factors for Glen's A

### Factor = 1.5

In [65]:
factor = 1.5
# I take 1879 because this is a year of length 0 in the ref
y_start = 1879

In [66]:
out = invert_parabolic_bed(gdir, glen_a=glen_a*factor, write=True)
# we keep the bed parabolic everywhere, just like for the inversion
cfg.PARAMS['bed_shape'] = 'parabolic'
flowline.init_present_time_glacier(gdir)
fls = gdir.read_pickle('model_flowlines')
u_fl = copy.deepcopy(fls[0])

In [67]:
# Make it flat
u_fl = copy.deepcopy(fls[0])
u_fl.bed_shape[60:] = 1e-5
u_fl.bed_shape[61] = 5e-5
u_fl.bed_shape[62] = 4e-5
u_fl.bed_shape[63] = 3e-5
u_fl.bed_shape[64] = 2e-5
u_fl.bed_shape[65:] = 1e-5

u_fl.bed_shape[60:] = 1e-5


In [57]:
# See how a bigger glacier looks like
mb_mod = massbalance.BackwardsMassBalanceModel(gdir, bias=200)
model = flowline.FluxBasedModel([copy.deepcopy(u_fl)], mb_model=mb_mod, y0=0., fs=0., glen_a=glen_a*factor)
model.run_until_equilibrium()
graphics.plot_modeloutput_map(gdir, model=model)

RuntimeError: Glacier exceeds domain boundaries.

In [68]:
# we now use the actual Histalp mass balance to run the model
# on the flat tongue
mb_mod = massbalance.HistalpMassBalanceModel(gdir)
model = flowline.FluxBasedModel([copy.deepcopy(u_fl)],
                                mb_model=mb_mod, y0=y_start, 
                                fs=0., glen_a=glen_a*factor)

In [70]:
# For every year where we have an observation,
# we are going to take the length of the glacier
ref_data = lengths_data.loc[y_start:].copy()
years = ref_data.index.values
length = years*0.
for i, y in enumerate(years):
    print(y)
    model.run_until(y)
    length[i] = model.length_m
# add oggm time series to the relative length
ref_data['oggm'] = length - length[0]
# Make a plot of it
ref_data.plot();
plt.axhline(0, color='k', linestyle=':');
plt.title('Factor: 1.5, Flad Bed');

1879


RuntimeError: Glacier exceeds domain boundaries.